In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [40]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from glob import glob
import jieba
import jieba.analyse

import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.model_selection import train_test_split  

def cuttrain_data(df):
  cuttitle = []
  for d in df['title'].astype(str):
    cut = jieba.analyse.extract_tags(d,topK=2)
    cuttitle.append(cut)
  print("in cuttitle")
  # for d in cuttitle:
  #   print(d)
  cutcontext = []
  for d in df['context']:
    cut = jieba.analyse.extract_tags(d,topK=2)
    cutcontext.append(cut)
  # print("in context")
  # for d in cutcontext:
  #   print(d)
  push = []
  for d in df['push']:
    push.append(d)
  down = []
  for d in df['down']:
    down.append(d)

  data = {'title':cuttitle ,'context':cutcontext ,'push':push ,'down':down}
  df1 = pd.DataFrame(data)

  df1['context'] = df1['context'].apply(lambda x : str(x)[1:-2].replace("'","").replace(",",""))
  # print("df1['context'] = ")
  # print(df1['context'])
  df1['title'] = df1['title'].apply(lambda x : str(x)[1:-2].replace("'","").replace(",",""))
  # print("df1['title'] = ")
  # print(df1['title'])
  df1['ans_push'] = df1['title'].astype(str)+' '+df1['context'].astype(str)+' '+df1['push'].astype(str)
  # print("df1['ans_push'] = ")
  # for d in df1['ans_push']:
  #   print(d)
  df1['ans_down'] = df1['title'].astype(str)+' '+df1['context'].astype(str)+' '+df1['down'].astype(str)
  # print("df1['ans_down'] = ")
  # for d in df1['ans_down']:
  #   print(d)
  return df1

def cuttest_data(df):
  cuttitle = []
  for d in df['title'].astype(str):
    cut = jieba.analyse.extract_tags(d,topK=2)
    cuttitle.append(cut)
  cutcontext = []
  for d in df['context']:
    cut = jieba.analyse.extract_tags(d,topK=10)
    cutcontext.append(cut)

  data = {'title':cuttitle ,'context':cutcontext}
  df1 = pd.DataFrame(data)

  df1['context'] = df1['context'].apply(lambda x : str(x)[1:-2].replace("'","").replace(",",""))
  df1['title'] = df1['title'].apply(lambda x : str(x)[1:-2].replace("'","").replace(",",""))
  df1['key'] = df1['title'].astype(str)+' '+df1['context'].astype(str)
  return df1

def count_push(df):
  vectorizer = CountVectorizer()
  X = vectorizer.fit_transform(df['ans_push'])
  tfidf = TfidfTransformer() 
  tf=tfidf.fit_transform(X)
  word = vectorizer.get_feature_names() #詞袋 
  return tf

def count_down(df):
  vectorizer = CountVectorizer()
  X = vectorizer.fit_transform(df['ans_down'])
  tfidf = TfidfTransformer() 
  tf=tfidf.fit_transform(X)
  word = vectorizer.get_feature_names() #詞袋 
  return tf

def count_test(df):
  vectorizer = CountVectorizer()
  X = vectorizer.transform(df['key'])
  tfidf = TfidfTransformer() 
  tf=tfidf.fit_transform(X)
  word = vectorizer.get_feature_names() #詞袋 
  return tf

def knn_push(x_data,y_data):
  X_train, X_test, y_train, y_test = train_test_split(x_data, y_data['push'].astype(str), test_size=0.1, random_state=0)
  clf=KNeighborsClassifier(n_neighbors= 70, p=2, weights= 'uniform')
  clf.fit(X_train,y_train)
  y_pred = clf.predict(X_test)
  ans_push = y_pred    # 預測模型
  print("test_p = clf.predict(tf_test) = ")
  print(len(test_p))
  return ans_push

def knn_down(x_data,y_data,):
  X_train, X_test, y_train, y_test = train_test_split(x_data, y_data['down'].astype(str), test_size=0.1, random_state=0)
  clf=KNeighborsClassifier(n_neighbors= 70, p=2, weights= 'uniform')
  clf.fit(X_train,y_train)
  y_pred = clf.predict(X_test)    # 預測模型

  ans_down = y_pred

  # print("精準度：",metrics.accuracy_score(y_test, y_pred))
  return ans_down

train_data = pd.read_csv("/content/drive/MyDrive/NatureLanguageProgram/NLP_final_project/train_data.csv")
train_label = pd.read_csv("/content/drive/MyDrive/NatureLanguageProgram/NLP_final_project/train_label.csv")
test_data = pd.read_csv("/content/drive/MyDrive/NatureLanguageProgram/NLP_final_project/test_data.csv")

df_list = [pd.read_csv(file) for file in files]  #串列中包含兩個Pandas DataFrame
total_train_data = pd.merge(df_list[0], df_list[1], on='index')


the_training_data = total_train_data[['title' ,'context' ,'push' ,'down']]
the_test_data = test_data[['title' ,'context']]


train = cuttrain_data(the_training_data)
print("len(train)")
print(len(train))


test = cuttest_data(the_test_data)
print("len(test)")
print(len(test))

tf_test = count_test(test)
print("len(tf_test)")
print(tf_test.shape)

tf_push = count_push(train)
tf_down = count_down(train)
print("len(tf_push)")
print(tf_push.shape)
print("len(tf_down)")
print(tf_down.shape)

X_train, X_test, y_train, y_test = train_test_split(tf_push, train['push'].astype(str), test_size=0.1, random_state=1)
clf=KNeighborsClassifier(n_neighbors=10, p=2, weights= 'uniform')
clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
ans_push = y_pred
test_p = clf.predict(tf_test)
print("test_p = clf.predict(tf_test) = ")
print(len(test_p))

X_train, X_test, y_train, y_test = train_test_split(tf_down, train['push'].astype(str), test_size=0.1, random_state=1)
clf=KNeighborsClassifier(n_neighbors=10, p=2, weights= 'uniform')
clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
ans_push = y_pred 
test_d = clf.predict(tf_test)
print("test_d = clf.predict(tf_test) = ")
print(len(test_d))

# ans_push,test_p = knn_push(tf_push,train,tf_test)

# ans_down,test_d = knn_down(tf_down,train,tf_test)

# test_index=[]
# for i in range(len(ans_push)):
#     test_index.append(i)
# data = {'index': test_index, 'push': ans_push, 'down': ans_down}

# export = pd.DataFrame(data)
# export.to_csv('my_submission.csv' ,index = False)

# my_answer= pd.read_csv("./my_submission.csv")
# my_answer

in cuttitle


KeyboardInterrupt: ignored